<a href="https://colab.research.google.com/github/krbdfirst/HealthFacilities-service-area/blob/main/Determine_Service_Area_of_HF_in_Nigeria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Installing Required Libraries
!pip install geopy geopandas osmnx openrouteservice folium shapely


In [ ]:
#Loading Health Facility DataSet using pandas(Source:GRID3)
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/DeepTech Ready/GDS Week 3/GRID3_NGA_health_facilities_v2_0_5806009649412052847 (1).csv")
df.head()

,OBJECTID,globalid,nhfr_uid,nhfr_facility_code,country,iso,state,lga,lga_name_disagreement,ward,...,ownership,ownership_type,facility_level,facility_level_option,latitude,longitude,geocoordinates_source,last_updated,x,y
0,1,9c2b58dd-1e99-49b9-970e-eabce1c7dd10,57132712.0,08/07/1/1/1/0030,Nigeria,NGA,Borno,Damboa,0,Gumsuri,...,Public,Local Government,Primary,Primary Health Center,11.059840,12.808550,NHFR_2024,2024-11-11,12.808550,11.059840
1,2,5a3864b5-d66b-4683-90cd-9c76af393450,42800186.0,08/02/1/1/1/0044,Nigeria,NGA,Borno,Askira/Uba,0,Uba,...,Public,Local Government,Primary,Primary Health Center,10.458997,13.213818,GRID3_EHEALTH,2024-11-11,13.213818,10.458997
2,3,cfe845eb-9f71-4b99-9603-d2f389a73ba0,26934297.0,08/02/1/1/1/0034,Nigeria,NGA,Borno,Askira/Uba,0,Ngulde,...,Public,Local Government,Primary,Primary Health Center,10.632343,12.545808,GRID3_EHEALTH,2024-11-11,12.545808,10.632343
3,4,0fcbdf4a-8ea2-4df9-9167-4c9f1377f27c,43109689.0,08/06/1/1/1/0021,Nigeria,NGA,Borno,Chibok,0,Shikarkir,...,Public,Local Government,Primary,Primary Health Center,10.802780,12.776800,GRID3_EHEALTH,2024-11-11,12.776800,10.802780
4,5,12c47bd1-535e-4de7-9cad-ad34f43b1965,28489340.0,08/06/1/1/1/0025,Nigeria,NGA,Borno,Chibok,0,Kuburmbula,...,Public,Local Government,Primary,Health Post,10.888690,12.903770,GRID3_EHEALTH,2024-11-11,12.903770,10.888690


In [ ]:
"""Importing Geopandas, for operating on dataset with spatial properties,
   Importing Shapely to define point data
"""

import geopandas as gpd
from shapely.geometry import Point

#covert df to geopandas
geometry = [Point(xy) for xy in zip(df["longitude"], df["latitude"])]
gdf = gpd.GeoDataFrame(df, crs="EPSG:4326", geometry=geometry)
gdf.head()

,OBJECTID,globalid,nhfr_uid,nhfr_facility_code,country,iso,state,lga,lga_name_disagreement,ward,...,ownership_type,facility_level,facility_level_option,latitude,longitude,geocoordinates_source,last_updated,x,y,geometry
0,1,9c2b58dd-1e99-49b9-970e-eabce1c7dd10,57132712.0,08/07/1/1/1/0030,Nigeria,NGA,Borno,Damboa,0,Gumsuri,...,Local Government,Primary,Primary Health Center,11.059840,12.808550,NHFR_2024,2024-11-11,12.808550,11.059840,POINT (12.80855 11.05984)
1,2,5a3864b5-d66b-4683-90cd-9c76af393450,42800186.0,08/02/1/1/1/0044,Nigeria,NGA,Borno,Askira/Uba,0,Uba,...,Local Government,Primary,Primary Health Center,10.458997,13.213818,GRID3_EHEALTH,2024-11-11,13.213818,10.458997,POINT (13.21382 10.459)
2,3,cfe845eb-9f71-4b99-9603-d2f389a73ba0,26934297.0,08/02/1/1/1/0034,Nigeria,NGA,Borno,Askira/Uba,0,Ngulde,...,Local Government,Primary,Primary Health Center,10.632343,12.545808,GRID3_EHEALTH,2024-11-11,12.545808,10.632343,POINT (12.54581 10.63234)
3,4,0fcbdf4a-8ea2-4df9-9167-4c9f1377f27c,43109689.0,08/06/1/1/1/0021,Nigeria,NGA,Borno,Chibok,0,Shikarkir,...,Local Government,Primary,Primary Health Center,10.802780,12.776800,GRID3_EHEALTH,2024-11-11,12.776800,10.802780,POINT (12.7768 10.80278)
4,5,12c47bd1-535e-4de7-9cad-ad34f43b1965,28489340.0,08/06/1/1/1/0025,Nigeria,NGA,Borno,Chibok,0,Kuburmbula,...,Local Government,Primary,Health Post,10.888690,12.903770,GRID3_EHEALTH,2024-11-11,12.903770,10.888690,POINT (12.90377 10.88869)


In [ ]:
#Connect to Open Route Services
import openrouteservice
api_key = "5b3ce3597851110001cf6248f3fad7ce93b6429b810aa041d84df4ec"
client = openrouteservice.Client(key=api_key)

coordinates = [[5.25, 13.05]]
isochrones = client.isochrones(
    locations = coordinates,
    profile = "driving-car",
    range_type = "time",
    range = [120,300]
)

/usr/local/lib/python3.11/dist-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/usr/local/lib/python3.11/dist-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/usr/local/lib/python3.11/dist-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/usr/local/lib/python3.11/dist-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/usr/local/lib/python3.11/dist-packages/openrouteservice/client.py:211: UserWarning: Rat

In [ ]:
from shapely.geometry import shape
import time

def get_service_area(longitude, latitude):
    try:
        isochrones = client.isochrones(
            locations=[[longitude, latitude]],  # Correct structure
            profile="driving-car",
            range_type="time",
            range=[120, 300]  # 2 min and 5 min
        )
        features = isochrones["features"]
        polygon_2km = shape(features[0]["geometry"]) if len(features) > 0 else None
        polygon_5km = shape(features[1]["geometry"]) if len(features) > 1 else None
        return polygon_2km, polygon_5km
    except Exception as e:
        time.sleep(61)
        print(f"Error for ({longitude}, {latitude}):", e)
        return None, None

In [ ]:
# Initialize lists
service_area_2km = []
service_area_5km = []

# Generate isochrones for each point
for _, row in gdf.iterrows():
    polygon_2km, polygon_5km = get_service_area(row["longitude"], row["latitude"])
    service_area_2km.append(polygon_2km)
    service_area_5km.append(polygon_5km)

# Add to GeoDataFrame
gdf["service_area_2km"] = service_area_2km
gdf["service_area_5km"] = service_area_5km

# Create new GeoDataFrames for plotting or further analysis
service_area_2km_gdf = gpd.GeoDataFrame(geometry=gdf["service_area_2km"], crs="EPSG:4326")
service_area_5km_gdf = gpd.GeoDataFrame(geometry=gdf["service_area_5km"], crs="EPSG:4326")

/usr/local/lib/python3.11/dist-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 7th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/usr/local/lib/python3.11/dist-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 8th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/usr/local/lib/python3.11/dist-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 9th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


Error for (6.755968333, 7.941426667): 403 ({'error': 'Quota exceeded'})
Error for (6.769446667, 7.13086): 403 ({'error': 'Quota exceeded'})
Error for (6.16992191515, 7.58812400998): 403 ({'error': 'Quota exceeded'})
Error for (7.09819, 7.20628): 403 ({'error': 'Quota exceeded'})
Error for (6.2428192962, 7.54817940138): 403 ({'error': 'Quota exceeded'})
Error for (7.554658333, 7.187435): 403 ({'error': 'Quota exceeded'})
Error for (7.6512797755, 7.16301913138): 403 ({'error': 'Quota exceeded'})
Error for (7.3249133, 5.012195): 403 ({'error': 'Quota exceeded'})
Error for (7.321581667, 4.997365): 403 ({'error': 'Quota exceeded'})
Error for (7.3092183, 5.0072833): 403 ({'error': 'Quota exceeded'})
Error for (3.646216667, 6.841305): 403 ({'error': 'Quota exceeded'})
Error for (7.884983333, 6.132526667): 403 ({'error': 'Quota exceeded'})
Error for (7.316343333, 5.02345): 403 ({'error': 'Quota exceeded'})
Error for (6.02039, 6.29418): 403 ({'error': 'Quota exceeded'})
Error for (6.85025, 12.6

In [ ]:
import folium

map = folium.Map(location=[gdf.longtude.mean(), ddf.latitude.mean()], zoom_start=10)

for _, row in gdf.iterrows():
  folium.Marker(
      location=[row["latitude"], row["longitude"]],
      popup=row["facility_name"],
      icon=folium.Icon(color="green", icon = "hospital")
  ).add_to(map)

for _, row in service_area_2km_gdf.iterrows():
  if row.geometry:
    folium.GeoJson(
        row.geometry,
        style_function=lambda x: {"fillColor": "green", "fillOpacity": 0.2}
    ).add_to(map)

for _, row in service_area_5km_gdf.iterrows():
  if row.geometry:
    folium.GeoJson(
        row.geometry,
        style_function=lambda x: {"fillColor": "red", "fillOpacity": 0.2}
    ).add_to(map)

map

In [ ]:
service_area_2km_gdf.to_file('/content/drive/MyDrive/DeepTech Ready/GDS Week 3/service_area_2km_gdf.geojson', driver='GeoJSON')
service_area_5km_gdf.to_file('/content/drive/MyDrive/DeepTech Ready/GDS Week 3/service_area_5km_gdf.geojson', driver='GeoJSON')